In [38]:
#Import Libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import webbrowser
import os
import plotly.graph_objects as go
from datetime import datetime
import pytz

In [23]:
#Load Datasets
apps_df = pd.read_csv('/content/Play Store Data.csv')
reviews_df = pd.read_csv('/content/User Reviews.csv')
global_df = pd.read_csv('/content/Global EV Data 2024.csv')

In [39]:
# Current IST time
ist = pytz.timezone('Asia/Kolkata')
current_time = datetime.now(ist)
current_hour = current_time.hour
print("Current IST Time:", current_time.strftime("%I:%M %p"))

Current IST Time: 02:32 PM


In [40]:
def clean_installs(x):
    try:
        return float(str(x).replace('+','').replace(',','').replace('M','000000').replace('k','000').strip())
    except:
        return np.nan

def clean_size(x):
    try:
        s = str(x).upper()
        if 'M' in s:
            return float(s.replace('M',''))
        elif 'K' in s:
            return float(s.replace('K',''))/1024
        else:
            return np.nan
    except:
        return np.nan

apps_df['Installs_Clean'] = apps_df['Installs'].apply(clean_installs)
apps_df['Size_MB'] = apps_df['Size'].apply(clean_size)
apps_df['Last_Updated'] = pd.to_datetime(apps_df['Last Updated'], errors='coerce')
apps_df['Last_Updated_Month'] = apps_df['Last_Updated'].dt.strftime('%B')


Task 1 : Grouped Bar Chart (3-5 PM)

In [42]:
if 15 <= current_hour < 17:
    df1 = apps_df[(apps_df['Rating']>=4.0) &
                  (apps_df['Size_MB']>=10) &
                  (apps_df['Last_Updated_Month']=="January")]
    category_stats = df1.groupby('Category').agg({
        'Rating':'mean',
        'Reviews':'sum',
        'Installs_Clean':'sum'
    }).reset_index()
    top10 = category_stats.nlargest(10,'Installs_Clean')

    fig1 = go.Figure()
    fig1.add_trace(go.Bar(name='Avg Rating', x=top10['Category'], y=top10['Rating'],
                          marker_color='#636EFA', text=top10['Rating'].round(2), textposition='auto'))
    fig1.add_trace(go.Bar(name='Total Reviews', x=top10['Category'], y=top10['Reviews'],
                          marker_color='#EF553B', text=top10['Reviews'].astype(int), textposition='auto', yaxis='y2'))
    fig1.update_layout(
        title='Top 10 Categories: Avg Rating & Review Count',
        xaxis=dict(title='Category', tickangle=-45),
        yaxis=dict(title='Average Rating', range=[0,5], titlefont=dict(color='#636EFA'), tickfont=dict(color='#636EFA')),
        yaxis2=dict(title='Total Reviews', overlaying='y', side='right', titlefont=dict(color='#EF553B'), tickfont=dict(color='#EF553B')),
        barmode='group', template="plotly_dark"
    )
    fig1.show()
else:
    print("✗ Task 1 hidden - Available 3–5 PM IST")

✗ Task 1 hidden - Available 3–5 PM IST


Task 2 : Choropleth Map (6-8 PM)

In [50]:
if 18 <= current_hour < 20:
    # Step 1: Filter apps_df categories (exclude A, C, G, S)
    df2 = apps_df[~apps_df['Category'].str.startswith(('A','C','G','S'))]
    top5_cats = df2.groupby('Category')['Installs_Clean'].sum().nlargest(5).index
    df2 = df2[df2['Category'].isin(top5_cats)]

    # Step 2: Merge with global_df to bring in Country information
    # Assuming both have "App" column for joining
    df2_geo = df2.merge(global_df, on="App", how="inner")

    # Step 3: Group by Category & Country
    df2_country = df2_geo.groupby(['Category','Country'])['Installs_Clean'].sum().reset_index()

    # Step 4: Choropleth visualization
    fig2 = go.Figure()
    for cat in df2_country['Category'].unique():
        df_cat = df2_country[df2_country['Category']==cat]
        fig2.add_trace(go.Choropleth(
            locations=df_cat['Country'],
            locationmode="country names",
            z=df_cat['Installs_Clean'],
            text=cat,
            colorscale="Viridis",
            colorbar_title="Installs",
            marker_line_width=np.where(df_cat['Installs_Clean']>1_000_000,2,0.5)
        ))

    fig2.update_layout(title="Installs by Category & Country (Task 2)", template="plotly_dark")
    fig2.show()

else:
    print("✗ Task 2 hidden - Available 6–8 PM IST")


✗ Task 2 hidden - Available 6–8 PM IST


Task 3 : Dual Axis Chart (1-2 PM)

In [51]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

if 13 <= current_hour < 14:
    df3 = apps_df.copy()
    # Extract Android version once as float for filtering
    df3['Android_Version'] = df3['Android Ver'].str.extract(r'(\d+\.\d+)')[0].astype(float)

    # Apply all filters
    df3 = df3[(df3['Installs_Clean']>=10000) &
              (df3['Revenue']>=10000) &
              (df3['Android_Version']>4.0) &
              (df3['Size_MB']>15) &
              (df3['Content Rating']=="Everyone") &
              (df3['App'].str.len()<=30)]

    top3_cats = df3.groupby('Category')['Installs_Clean'].sum().nlargest(3).index
    df3 = df3[df3['Category'].isin(top3_cats)]

    agg = df3.groupby(['Category','Type'], as_index=False).agg({
        'Installs_Clean':'mean',
        'Revenue':'mean'
    })

    # Create subplots with secondary y-axis
    fig3 = make_subplots(specs=[[{"secondary_y": True}]])

    # Add bar trace for average installs
    fig3.add_trace(
        go.Bar(x=agg['Category'] + " - " + agg['Type'], y=agg['Installs_Clean'], name="Avg Installs"),
        secondary_y=False)

    # Add scatter trace for revenue on secondary y-axis
    fig3.add_trace(
        go.Scatter(x=agg['Category'] + " - " + agg['Type'], y=agg['Revenue'],
                   name="Avg Revenue", mode="lines+markers"),
        secondary_y=True)

    # Update layout and axis titles
    fig3.update_layout(
        title="Dual Axis: Installs vs Revenue (Task 3)",
        xaxis_title="Category-Type",
        template="plotly_dark"
    )
    fig3.update_yaxes(title_text="Avg Installs", secondary_y=False)
    fig3.update_yaxes(title_text="Avg Revenue", secondary_y=True)

    fig3.show()
else:
    print("✗ Task 3 hidden - Available 1–2 PM IST")


✗ Task 3 hidden - Available 1–2 PM IST


Task 4 : Time Series Line Chart (6-9 PM)

In [52]:
if 18 <= current_hour < 21:
    df4 = apps_df.copy()
    # Apply all filters
    df4 = df4[(df4['Reviews'] > 500) &
              (~df4['App'].str[0].str.lower().isin(['x','y','z'])) &
              (~df4['App'].str.contains('S', case=False)) &
              (df4['Category'].str.startswith(('E','C','B')))]

    # Translate categories
    translations = {'Beauty': 'ब्यूटी', 'Business': 'வணிகம்', 'Dating': 'Dating in German'}
    df4['Category_Translated'] = df4['Category'].replace(translations)

    # Convert dates and aggregate installs monthly
    df4['Month'] = pd.to_datetime(df4['Last_Updated']).dt.to_period('M')
    df_agg = df4.groupby(['Month', 'Category_Translated'])['Installs_Clean'].sum().reset_index()

    # Calculate Monthly MoM Growth %
    df_agg['MoM_Growth'] = df_agg.groupby('Category_Translated')['Installs_Clean'].pct_change() * 100

    fig4 = go.Figure()
    for cat in df_agg['Category_Translated'].unique():
        df_cat = df_agg[df_agg['Category_Translated'] == cat]
        fig4.add_trace(go.Scatter(
            x=df_cat['Month'].dt.to_timestamp(),
            y=df_cat['Installs_Clean'],
            mode='lines+markers',
            name=cat
        ))
        # Shade area under curve for growth > 20%
        high_growth = df_cat[df_cat['MoM_Growth'] > 20]
        fig4.add_trace(go.Scatter(
            x=high_growth['Month'].dt.to_timestamp(),
            y=high_growth['Installs_Clean'],
            mode='none',
            fill='tozeroy',
            fillcolor='rgba(255,0,0,0.2)',
            showlegend=False,
            hoverinfo='skip'
        ))

    fig4.update_layout(
        title="Total Installs Over Time by Category (Task 4)",
        xaxis_title="Month",
        yaxis_title="Total Installs",
        template="plotly_dark"
    )
    fig4.show()
else:
    print("✗ Task 4 hidden - Available 6–9 PM IST")


✗ Task 4 hidden - Available 6–9 PM IST


Task 5 : Bubble Chart (5-7 PM)

In [53]:
if 17 <= current_hour < 19:
    df5 = apps_df.copy()
    df5 = df5[(df5['Rating'] > 3.5) & (df5['Reviews'] > 500)]
    categories_allowed = ['Game', 'Beauty', 'Business', 'Comics', 'Communication', 'Dating', 'Entertainment', 'Social', 'Event']
    df5 = df5[df5['Category'].isin(categories_allowed)]
    df5 = df5[~df5['App'].str.contains('S', case=False)]

    # Merge to get sentiment subjectivity
    df5 = df5.merge(reviews_df[['App', 'Sentiment_Subjectivity']], on='App', how='left')
    df5 = df5[df5['Sentiment_Subjectivity'] > 0.5]
    df5 = df5[df5['Installs_Clean'] > 50000]

    translations = {'Beauty': 'ब्यूटी', 'Business': 'வணிகம்', 'Dating': 'Dating in German'}
    df5['Category_Translated'] = df5['Category'].replace(translations)

    fig5 = go.Figure()
    for cat in df5['Category_Translated'].unique():
        df_cat = df5[df5['Category_Translated'] == cat]
        marker_color = 'pink' if cat == 'Game' else 'blue'
        fig5.add_trace(go.Scatter(
            x=df_cat['Size_MB'], y=df_cat['Rating'],
            mode='markers', name=cat,
            marker=dict(
                size=df_cat['Installs_Clean'] / 100000,
                color=marker_color,
                sizemode='area',
                sizeref=2. * max(df5['Installs_Clean'] / 100000) / (100 ** 2),
                sizemin=4
            ),
            text=df_cat['App']
        ))

    fig5.update_layout(title='App Size vs Rating Bubble Chart (Task 5)', template='plotly_dark',
                       xaxis_title='App Size (MB)', yaxis_title='Average Rating')
    fig5.show()
else:
    print("✗ Task 5 hidden - Available 5–7 PM IST")


✗ Task 5 hidden - Available 5–7 PM IST


Task 6 : Stacked Area Chart (4-6 PM)

In [54]:
if 16 <= current_hour < 18:
    df6 = apps_df.copy()
    df6 = df6[(df6['Rating'] >= 4.2) &
              (~df6['App'].str.contains(r'\d', regex=True)) &
              (df6['Category'].str.startswith(('T','P'))) &
              (df6['Reviews'] > 1000) &
              (df6['Size_MB'].between(20,80))]

    translations = {'Travel & Local':'Voyage', 'Productivity':'Productividad', 'Photography':'写真'}
    df6['Category_Translated'] = df6['Category'].replace(translations)

    df6['Month'] = pd.to_datetime(df6['Last_Updated'], errors='coerce').dt.to_period('M')
    df_agg6 = df6.groupby(['Month','Category_Translated'])['Installs_Clean'].sum().reset_index()

    # Calculate cumulative installs per category
    df_agg6['Cumulative_Installs'] = df_agg6.groupby('Category_Translated')['Installs_Clean'].cumsum()

    # Calculate MoM growth %
    df_agg6['MoM_Growth'] = df_agg6.groupby('Category_Translated')['Installs_Clean'].pct_change()*100

    fig6 = go.Figure()
    for cat in df_agg6['Category_Translated'].unique():
        df_cat = df_agg6[df_agg6['Category_Translated'] == cat]
        fig6.add_trace(go.Scatter(
            x=df_cat['Month'].dt.to_timestamp(),
            y=df_cat['Cumulative_Installs'],
            mode='lines',
            stackgroup='one',
            name=cat
        ))
        # Highlight periods with >25% MoM growth
        high_growth = df_cat[df_cat['MoM_Growth'] > 25]
        fig6.add_trace(go.Scatter(
            x=high_growth['Month'].dt.to_timestamp(),
            y=high_growth['Cumulative_Installs'],
            mode='none',
            fill='tonexty',
            fillcolor='rgba(255, 0, 0, 0.3)',
            showlegend=False,
            hoverinfo='skip'
        ))

    fig6.update_layout(
        title="Cumulative Installs by Category (Task 6)",
        template="plotly_dark",
        xaxis_title="Month",
        yaxis_title="Cumulative Installs"
    )
    fig6.show()
else:
    print("✗ Task 6 hidden - Available 4–6 PM IST")


✗ Task 6 hidden - Available 4–6 PM IST
